In [1]:
install.packages("slider")
# raw file name: Developing-SVEIR-model-in-R.html


# LOADING LIBRARIES
## Data wrangling, 모형 구축 ??? ???각화??? ????????? R ??????지??? 불러?????? 과정???므???, 무시????????? ?????????.
library(tidyverse)
library(lubridate)
library(vroom)
library(rlist) # for list to tibble combined by rows
library(deSolve)
library(Cairo)
library(scales) # for the comma formatting

ggplot2::theme_set(theme_classic())

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘scales’


The following object is masked from ‘package:vroom’:

    col_factor


The following object is masked from ‘package:purrr’:

    discard


The following object is masked from ‘package:readr’:

    col_factor




In [8]:
path <- vroom("../../../../00_path/SVEIR/sveir_path.csv")
input_path <- path$input
input_path

output_path <- path$output
output_path

New names:
* `` -> ...1

Rows: 1 Columns: 3

── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): input, output
dbl (1): ...1


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



[1] "../../../../01_data/SVEIR/sveir_input_data.csv"

[1] "../../../../03_result/SVEIR/211119/sveir_211119.csv"

In [21]:




# LOADING AND WRANGLING DATA
## "코로???19 감염??????_~20210701.csv"???료는 OPEN API??? 부??? ???보한 ???료에 ???????????????.-> open api??? ????????? 매일 ???????????? 가????????????!!!
## ????????? ???보한 ???료로부??? ??? ???진자 ??? 계산 ??? 7-MA(7??? ????????????)??? 계산?????? 과정?????????. 
## 최종????????? 모델??? ??? ???각화??? ?????? ???료는 case2??? ???????????????.

case <- vroom("../../../../01_data/SVEIR/sveir_input_data.csv")
case2 <- case %>% 
    distinct() %>% # 중복??? ??????
    arrange(stateDt) %>% # 발표??? 기??? ???름차??? ??????
    select(stateDt, decideCnt, clearCnt, deathCnt) %>% 
    mutate(stateDt = ymd(stateDt)) %>%
    filter(stateDt != ymd("20200101")) %>%
    filter(stateDt >= ymd("20200206")) %>% # 2020/02/06 ?????? ???료는 불안????????? ??????
    mutate(daily_confirmed = c(5, diff(decideCnt))) %>%  # diff()??? ?????? ?????? ???진자 ??? 계산
    rename(confirmed = decideCnt) %>%
    mutate( # 7??? ???????????? 추???.
        daily_MA = slider::slide_dbl(daily_confirmed, mean, .before = 3, .after = 3, .complete = TRUE),
        daily_MA_clearCnt = slider::slide_dbl(clearCnt, mean, .before = 3, .after = 3, .complete = TRUE),
        daily_MA_deathCnt = slider::slide_dbl(deathCnt, mean, .before = 3, .after = 3, .complete = TRUE)
    ) %>% 
    filter(!is.na(daily_MA))

# ???료에??? 모형 구축??? ?????? 간략??? 변??? ???명??? ????????? 같습??????:

## stateDt: 발표 기?????? (?????????)
## decideCnt: ?????? ???진자 ??? (?????????)
## deathCnt: ?????? ???망자 ??? (?????????)
## clearCnt: ?????? 격리?????? ??? (?????????)
## daily_confirmed: ??? ???진자 ??? (decideCnt??? 계산, 추??? ??????:2020/02/06 ???진자 ??? 5???)
## daily_MA: ??? ???진자 ?????? 7-MA
## daily_MA_clearCnt: ?????? 격리?????? ?????? 7-MA
## daily_MA_deathCnt: ?????? ???망자 ?????? 7-MA

# 그리???, ??? 코드블럭?????? 마???막에 ????????? filter(!is.na(daily_MA))??? 경우, 
# 7-MA??? ???????????? ???료의 ?????? 3?????? 결측??? 발생????????? ??? 부분을 ???거한 것이?????? 보시면됩??????. 
# ???, 2020/02/06~2021/07/01 ???료에??? ????????? 3?????? ???거한 것입??????.


# MAKE A SVIR MODEL FUNCTION
## ??? 백신 1차접??? ?????? ?????? 30만명?????? 가??????????????????:
SVEIR <- function(time, state, parameters){
    with(as.list(c(state, parameters)),{
        N = S+I+R
        lambda = beta*(I/N)
        dS = -lambda*S - nu*300000*alpha
        dV = nu*300000*alpha 
        dE = lambda*S - sigma*E
        dI = sigma*E - gamma*I
        dR = gamma*I
        
        return(list(c(dS, dV, dE, dI, dR)))
    }
    )
}

# MODEL INPUTS
# ????????? 모형??? ???측에 ????????? 초깃??? ?????? 과정?????????.
## N: 2021??? 중위?????? 추계(참고)
## V(0): 7??? 1???까??? 1??? ?????? ?????? ???*0.8(참고: 매일 ???????????? ???)
## I(0): ??? ???진자 ???(daily_confirmed)??? 7-MA??? 마?????? ?????? 값을 초기 감염??? ?????? ??????
## E(0): I(0)??? 가???
## R(0): 격리??????(clearCnt) ??? ???망자 ???(deathCnt) 각각??? 7-MA 계산 ???, ??? 변?????? 마?????? ?????? 값의 ?????? 초기 ???복군 ?????? ??????
## S(0): N???V(0)???E(0)???I(0)???R(0)

N <- 51822000
v <- 15340827*0.7
i <- case2 %>% pull(daily_MA) %>% tail(1)
e <- i
r <- case2 %>% pull(daily_MA_clearCnt) %>% tail(1) + case2 %>% pull(daily_MA_deathCnt) %>% tail(1)
init <- c(S = N-i-e-r-v, V = v, E = e, I = i, R = r)

# ????????? 모형??? 모수?????? 관??? ???????????????. 
## Rp(prespecified reproduction number)??? 첨?????? ?????? ????????? specification?????? 0.8,0.9,1,1.1,1.2??? ???????????? ?????????, 
## β??? γ값에 ?????? ????????? 같이 결정?????????. 
## ?????? ??????, Rp=0.8??? 경우 γ=1/30???므??? ???????????? 감염??? β=1.2/30?????? 결정??? ?????????. 
## ?????? ?????? 로직?????? ????????? 같이 5개의 모수 ???????????? ??????리오??? 구성???????????????.

r0_1.2 <- c(gamma = 1/30, beta = 1.2/30, sigma = 1/14, nu = 1/14, alpha = 0.7)
r0_1.1 <- c(gamma = 1/30, beta = 1.1/30, sigma = 1/14, nu = 1/14, alpha = 0.7)
r0_1.0 <- c(gamma = 1/30, beta = 1.0/30, sigma = 1/14, nu = 1/14, alpha = 0.7)
r0_0.9 <- c(gamma = 1/30, beta = 0.9/30, sigma = 1/14, nu = 1/14, alpha = 0.7)
r0_0.8 <- c(gamma = 1/30, beta = 0.8/30, sigma = 1/14, nu = 1/14, alpha = 0.7)
r0 <- list(r0_1.2, r0_1.1, r0_1.0, r0_0.9, r0_0.8)

# ????????? 본격?????? 모형 ?????? 과정?????????. ??? 부분????????? 몬드리안AI?????? ??????????????? ????????? ???????????? 진행????????? ??? ??? 같습??????. 
# ??? 부분의 SVEIR 모형??? ?????? ?????? 만드??? 과정, 초깃??? ??????, 모수 ?????? ????????? 참고????????? ??? ??? 같습??????. 
# 참고???, ?????? 코드 블럭?????? ode()??????가 R??? {deSolve} ??????지??? ???????????? ???미분 방정???(ordinary differential equations, ODE)??? ?????????주는 ???????????????.

names(r0) <- str_c("R0_", seq(1.2, 0.8, -0.1))
SVEIR_korea <- vector(mode = "list", length = length(r0))
names(SVEIR_korea) <- names(r0)

PREDICT_LENGTH <- 120 # ?????? 코드?????? ?????? 부분입??????! ???측하??? ?????? ???짜로 바꾸??? ?????????.


for(i in seq_along(r0)){
  SVEIR_korea[[i]] <- ode(y = init, func = SVEIR, parms = r0[[i]], time = 1:PREDICT_LENGTH) %>%
    as_tibble %>%
    pivot_longer(S:R, names_to = "State") %>%
    mutate(value = as.numeric(value),
           time = as.numeric(time)) %>%
    arrange(State) %>%
    mutate(
      Date = rep(seq(ymd("2021-10-06"), by = "day", length.out = PREDICT_LENGTH), 5) #06-28
    ) %>%
    filter(time != 1)
}
# ???진님 ????????? 코드..????????????

# for(i in seq_along(r0)){
#   SVEIR_korea[[i]] <- ode(y = init, func = SVEIR, parms = r0[[i]], time = 1:PREDICT_LENGTH+1) %>%
#     as_tibble %>%
#     pivot_longer(S:R, names_to = "State") %>%
#     mutate(value = as.numeric(value),
#            time = as.numeric(time)) %>%
#     arrange(State) %>%
#     mutate(
#       Date = rep(seq(ymd("2021-07-12"), by = "day", length.out = PREDICT_LENGTH+1), 5)
#     ) %>%
#     filter(time != 1)
# }
# 
# SVEIR_k <-
#   bind_rows(SVEIR_korea[[1]], SVEIR_korea[[2]], SVEIR_korea[[3]],
#             SVEIR_korea[[4]], SVEIR_korea[[5]]) %>%
#     mutate(R = factor(rep(seq(1.2, 0.8, by = -0.1), each = PREDICT_LENGTH*5))) %>%
#     filter(State == "I")

SVEIR_k <-
  bind_rows(SVEIR_korea[[1]], SVEIR_korea[[2]], SVEIR_korea[[3]],
            SVEIR_korea[[4]], SVEIR_korea[[5]]) %>%
  mutate(R = factor(rep(seq(1.2, 0.8, by = -0.1), each =( PREDICT_LENGTH-1)*5))) %>%
  filter(State == "I")


SVEIR_k %>% write.csv("../../../../03_result/SVEIR/211102/sveir_211102.csv")



# VISUALIZATION
case2 %>%
  mutate(stateDt=case2$stateDt+3) %>%
  ggplot() +
  geom_line(aes(x = stateDt, y = daily_MA), col = "grey") +
  geom_line(data = SVEIR_k,
            aes(x = Date, y = value, col = R)) +
  scale_x_date(date_breaks = "1 months",
               date_labels = "%Y-%m") +
  scale_y_continuous(label = comma) +
  theme(
    axis.text.x = element_text(angle = 60, hjust = 1),
    legend.position = "bottom"
  ) +
  labs(
    x = "",
    y = "Daily confirmed cases",
    col = "Prespecified reproduction number"
  )

# past_df <- case %>%
#   select(stateDt, daily_confirmed, deathCnt, examCnt) %>%
#   mutate(?????????구분 = 'Reported') %>%
#   rename(??????=stateDt) %>%
#   rename(???망자???=deathCnt) %>%
#   rename(발생??????=daily_confirmed) %>%
#   rename(검??????=examCnt) %>%
#   pivot_longer(cols=c(발생??????, ???망자???, 검??????), names_to="variable", values_to="집계") %>%
#   arrange(variable)
# past_df

# future_df <- SVEIR_k %>%
#   rename(?????????구분=R) %>%
#   rename(집계="value") %>%
#   rename(??????=Date) %>%
#   mutate(variable='발생??????') %>%
#   select(??????, ?????????구분, variable, 집계)
# future_df

# new_data2 <- rbind(past_df, future_df) # column ?????? 주의

# new_data2 <- new_data2 %>%
#   mutate(???측모???='SVEIR') %>%
#   mutate(마스????????????부="O") %>%
#   mutate(?????????거리??????=0) %>%
#   mutate(백신???종률=0) %>%
#   mutate(지???='??????') %>%
#   mutate(??????='??????') %>%
#   mutate(???군구="??????") %>%
#   mutate(???가="??????민국")

# # new_data2 %>% write.csv("covid/project_1/data_2/20210713_SVEIR.csv")

# # new_data3 <- rbind(new_data, new_data2)
# # new_data3 %>% write.csv("covid/project_1/data_2/20210713_SVEIR_SEIR.csv")
# # new_data3
# #  new_data %>% tail(10)

# # case2 %>% tail(10)

# # new_data %>%  filter(!is.na(집계))

# # future_df


# # SVEIR_k


# # SVEIR_kk <- bind_rows(SVEIR_korea[[1]], SVEIR_korea[[2]], SVEIR_korea[[3]],
# #                      SVEIR_korea[[4]], SVEIR_korea[[5]]) %>%
# #   mutate(R = factor(rep(seq(1.2, 0.8, by = -0.1), each = (predict_length-1)*5)))



New names:
* `` -> ...1

Rows: 751 Columns: 5

── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
dbl  (4): ...1, decideCnt, deathCnt, clearCnt
date (1): stateDt


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



ERROR: Error: Problem with `mutate()` column `daily_MA`.
[34mℹ[39m `daily_MA = slider::slide_dbl(...)`.
[31m✖[39m there is no package called ‘slider’


In [20]:
# LOADING AND WRANGLING DATA

case <- vroom("../../../../01_data/SVEIR/sveir_input_data.csv")
case2 <- case %>% 
    distinct() %>% # 중복??? ??????
    arrange(stateDt) %>% # 발표??? 기??? ???름차??? ??????
    select(stateDt, decideCnt, clearCnt, deathCnt) %>% 
    mutate(stateDt = ymd(stateDt)) %>%
    filter(stateDt != ymd("20200101")) %>%
    filter(stateDt >= ymd("20200206")) %>% # 2020/02/06 ?????? ???료는 불안????????? ??????
    mutate(daily_confirmed = c(5, diff(decideCnt))) %>%  # diff()??? ?????? ?????? ???진자 ??? 계산
    rename(confirmed = decideCnt) %>%
    mutate( # 7??? ???????????? 추???.
        daily_MA = slider::slide_dbl(daily_confirmed, mean, .before = 3, .after = 3, .complete = TRUE),
        daily_MA_clearCnt = slider::slide_dbl(clearCnt, mean, .before = 3, .after = 3, .complete = TRUE),
        daily_MA_deathCnt = slider::slide_dbl(deathCnt, mean, .before = 3, .after = 3, .complete = TRUE)
    ) %>% 
    filter(!is.na(daily_MA))





New names:
* `` -> ...1

Rows: 751 Columns: 5

── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
dbl  (4): ...1, decideCnt, deathCnt, clearCnt
date (1): stateDt


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



ERROR: Error: Problem with `mutate()` column `daily_MA`.
[34mℹ[39m `daily_MA = slider::slide_dbl(...)`.
[31m✖[39m there is no package called ‘slider’


In [ ]:
# MAKE A SVIR MODEL FUNCTION
SVEIR <- function(time, state, parameters){
    with(as.list(c(state, parameters)),{
        N = S+I+R
        lambda = beta*(I/N)
        dS = -lambda*S - nu*300000*alpha
        dV = nu*300000*alpha 
        dE = lambda*S - sigma*E
        dI = sigma*E - gamma*I
        dR = gamma*I
        
        return(list(c(dS, dV, dE, dI, dR)))
    }
    )
}


N <- 51822000
v <- 15340827*0.7
i <- case2 %>% pull(daily_MA) %>% tail(1)
e <- i
r <- case2 %>% pull(daily_MA_clearCnt) %>% tail(1) + case2 %>% pull(daily_MA_deathCnt) %>% tail(1)
init <- c(S = N-i-e-r-v, V = v, E = e, I = i, R = r)


r0_1.2 <- c(gamma = 1/30, beta = 1.2/30, sigma = 1/14, nu = 1/14, alpha = 0.7)
r0_1.1 <- c(gamma = 1/30, beta = 1.1/30, sigma = 1/14, nu = 1/14, alpha = 0.7)
r0_1.0 <- c(gamma = 1/30, beta = 1.0/30, sigma = 1/14, nu = 1/14, alpha = 0.7)
r0_0.9 <- c(gamma = 1/30, beta = 0.9/30, sigma = 1/14, nu = 1/14, alpha = 0.7)
r0_0.8 <- c(gamma = 1/30, beta = 0.8/30, sigma = 1/14, nu = 1/14, alpha = 0.7)
r0 <- list(r0_1.2, r0_1.1, r0_1.0, r0_0.9, r0_0.8)


names(r0) <- str_c("R0_", seq(1.2, 0.8, -0.1))
SVEIR_korea <- vector(mode = "list", length = length(r0))
names(SVEIR_korea) <- names(r0)

PREDICT_LENGTH <- 120 
for(i in seq_along(r0)){
  SVEIR_korea[[i]] <- ode(y = init, func = SVEIR, parms = r0[[i]], time = 1:PREDICT_LENGTH) %>%
    as_tibble %>%
    pivot_longer(S:R, names_to = "State") %>%
    mutate(value = as.numeric(value),
           time = as.numeric(time)) %>%
    arrange(State) %>%
    mutate(
      Date = rep(seq(ymd("2021-10-06"), by = "day", length.out = PREDICT_LENGTH), 5) #06-28
    ) %>%
    filter(time != 1)
}

# for(i in seq_along(r0)){
#   SVEIR_korea[[i]] <- ode(y = init, func = SVEIR, parms = r0[[i]], time = 1:PREDICT_LENGTH+1) %>%
#     as_tibble %>%
#     pivot_longer(S:R, names_to = "State") %>%
#     mutate(value = as.numeric(value),
#            time = as.numeric(time)) %>%
#     arrange(State) %>%
#     mutate(
#       Date = rep(seq(ymd("2021-07-12"), by = "day", length.out = PREDICT_LENGTH+1), 5)
#     ) %>%
#     filter(time != 1)
# }
# 
# SVEIR_k <-
#   bind_rows(SVEIR_korea[[1]], SVEIR_korea[[2]], SVEIR_korea[[3]],
#             SVEIR_korea[[4]], SVEIR_korea[[5]]) %>%
#     mutate(R = factor(rep(seq(1.2, 0.8, by = -0.1), each = PREDICT_LENGTH*5))) %>%
#     filter(State == "I")

SVEIR_k <-
  bind_rows(SVEIR_korea[[1]], SVEIR_korea[[2]], SVEIR_korea[[3]],
            SVEIR_korea[[4]], SVEIR_korea[[5]]) %>%
  mutate(R = factor(rep(seq(1.2, 0.8, by = -0.1), each =( PREDICT_LENGTH-1)*5))) %>%
  filter(State == "I")


SVEIR_k %>% write.csv("../../../../03_result/SVEIR/211102/sveir_211102.csv")